In [1]:
args_pre = {
    'det_config': 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py',
    'det_checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth',
    'hand_detector_config': 'configs/hand/2d_kpt_sview_rgb_img/topdown_heatmap/coco_wholebody_hand/hrnetv2_w18_coco_wholebody_hand_256x256.py',
    'hand_detector_checkpoint': 'https://download.openmmlab.com/mmpose/hand/hrnetv2/hrnetv2_w18_coco_wholebody_hand_256x256-1c028db7_20210908.pth',
    'pose_detector_config': 'configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/hrnet_w48_coco_wholebody_256x192.py',
    'pose_detector_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_256x192-643e18cb_20200922.pth',
    'hand_lifter_config': 'configs/hand/3d_kpt_sview_rgb_vid/hand_pose_lift/handpose3d_dex_27frames_fullconv_supervised.py',
    'hand_lifter_checkpoint': '/home/ubuntu/PoseEstimation/mmpose/tools/work_dirs/handpose3d_dex_27frames_fullconv_supervised/epoch_80.pth',
    # Flags/Optional
    # 'video_path': 'demo/resources/jeldwen-1.mp4',
    'rebase_keypoint_height': True,
    'norm_pose_2d': None,
    'num_instances': -1,
    'show': False,
    'out_video_root': 'vis_results',
    'device': 'cuda:0',
    'det_cat_id': 1,
    'bbox_thr': 0.9,
    'kpt_thr': 0.3,
    'use_oks_tracking': None,
    'tracking_thr': 0.3,
    'euro': None,
    'radius': 8,
    'thickness': 2,
}
from types import SimpleNamespace
args = SimpleNamespace(**args_pre)

In [2]:
# Copyright (c) OpenMMLab. All rights reserved.
import copy
import os
import os.path as osp
from argparse import ArgumentParser

import time
import cv2
import mmcv
import numpy as np
import json
import pickle
import matplotlib.pyplot as plt

from mmpose.core.visualization import imshow_bboxes, imshow_keypoints
from mmpose.apis import (extract_pose_sequence, get_track_id,
                         inference_pose_lifter_model,
                         inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_3d_pose_result)

try:
    from mmdet.apis import inference_detector, init_detector

    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False
    
from demo import zach_v_body3d_two_stage_video as m_2d_3d_model 

In [3]:
LWRIST_IDX = 9
RWRIST_IDX = 10

skeleton = [[0, 1], [1, 2], [2, 3], [3, 4], 
            [0, 5], [5, 6], [6, 7], [7, 8], 
            [0, 9], [9, 10], [10, 11], [11, 12], 
            [0, 13], [13, 14], [14, 15], [15, 16], 
            [0, 17], [17, 18], [18, 19], [19, 20]
           ]

def create_writer(frame, args, fps):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(
        osp.join(args.out_video_root, f'tumeke_testing/vis_hands_2d_{osp.basename(args.file_path)}'),
        fourcc,
        fps,
        (frame.shape[1], frame.shape[0])
    )
    return writer

def process_image_hands(args, img_path):
    frame_idx = 0
    frame = cv2.imread(img_path)

    pose_det_model = init_pose_model(
        args.pose_detector_config,
        args.pose_detector_checkpoint,
        device=args.device.lower()
    )
    
    pose_det_results, _ = inference_top_down_pose_model(
        pose_det_model,
        frame,
        None,
        bbox_thr=None,
        format='xyxy',
        dataset=None,
        return_heatmap=False,
        outputs=None
    )
    rendered = imshow_keypoints(frame, np.expand_dims(pose_det_results[0]["keypoints"], 0), pose_kpt_color=[(255, 0, 0)]*21)
    plt.imshow(rendered)
    plt.show()
    print(pose_det_results)

def process_video_hands(args):
    frame_idx = 0
    assert has_mmdet, 'Please install mmdet to run the demo.'

#     args = parser.parse_args()
    assert args.show or (args.out_video_root != '')
    assert args.det_config is not None
    assert args.det_checkpoint is not None

    video = mmcv.VideoReader(args.file_path)
    assert video.opened, f'Failed to load video file {args.file_path}'

    # First stage: 2D pose detection
    print('Stage #1: 2D pose detection.')
    
    if args.out_video_root == '':
        save_out_video = False
    else:
        os.makedirs(args.out_video_root, exist_ok=True)
        save_out_video = True

    if save_out_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = video.fps
        writer = None
        
    person_det_model = init_detector(
        args.det_config, args.det_checkpoint, device=args.device.lower()
    )

    pose_det_model = init_pose_model(
        args.pose_detector_config,
        args.pose_detector_checkpoint,
        device=args.device.lower()
    )
    
    hand_det_model = init_pose_model(
        args.hand_detector_config,
        args.hand_detector_checkpoint,
        device=args.device.lower()
    )
    should_render_2d = True
    print("Initialized Model")
    
    num_instances = args.num_instances
    pose_det_dataset = pose_det_model.cfg.data['test']['type']
    pose_det_results_list = []
    hand_det_results_list = []
    writer = None
    if (args.detections_hand_2d == ""):
        for frame in video:
            t = time.time()
            mmdet_results = inference_detector(person_det_model, frame)

            # keep the person class bounding boxes.
            person_det_results = process_mmdet_results(mmdet_results,
                                                       args.det_cat_id)

            pose_det_results, _ = inference_top_down_pose_model(
                pose_det_model,
                frame,
                person_det_results,
                bbox_thr=args.bbox_thr,
                format='xyxy',
                dataset=pose_det_dataset,
                return_heatmap=False,
                outputs=None)

            if (should_render_2d):
                if (len(pose_det_results) == 0):
                    if (writer is None):
                        writer = create_writer(frame, args, video.fps)
                    writer.write(frame)
                    continue
            right_hand = pose_det_results[0]["keypoints"][112:112+21]
            #print(right_hand.shape, np.max(right_hand[:, 0]) + 10, frame.shape[1])
            x_min = int(max(np.min(right_hand[:, 0]) - 30, 0))
            x_max = int(min(np.max(right_hand[:, 0]) + 30, frame.shape[1]))
            y_min = int(max(np.min(right_hand[:, 1]) - 30, 0))
            y_max = int(min(np.max(right_hand[:, 1]) + 30, frame.shape[0]))
            
            left_hand = pose_det_results[0]["keypoints"][91:91+21]
            x_min_l = int(max(np.min(left_hand[:, 0]) - 30, 0))
            x_max_l = int(min(np.max(left_hand[:, 0]) + 30, frame.shape[1]))
            y_min_l = int(max(np.min(left_hand[:, 1]) - 30, 0))
            y_max_l = int(min(np.max(left_hand[:, 1]) + 30, frame.shape[0]))
    
            #hand_results = [{'bbox': np.array([x_min, y_min, bbox_size, bbox_size, 1.0])}]

            hand_det_results_r, _ = inference_top_down_pose_model(
                hand_det_model,
                frame[y_min:y_max, x_min:x_max],
                None,
                bbox_thr=None,
                format='xyxy',
                dataset=pose_det_dataset,
                return_heatmap=False,
                outputs=None)

            hand_det_results_l, _ = inference_top_down_pose_model(
                hand_det_model,
                frame[y_min_l:y_max_l, x_min_l:x_max_l],
                None,
                bbox_thr=None,
                format='xyxy',
                dataset=pose_det_dataset,
                return_heatmap=False,
                outputs=None)
            
            if (len(hand_det_results_l) > 1):
                print("More than 1 hand detected")
            if (should_render_2d):
                rendered = imshow_keypoints(
                    frame[y_min:y_max, x_min:x_max],
                    np.expand_dims(hand_det_results_r[0]["keypoints"], 0),
                    pose_kpt_color=[(0, 0, 255)]*21,
                    radius=4,
                    skeleton=skeleton,
                    pose_link_color=[(0, 0, 0)]*20
                )
                rendered_l = imshow_keypoints(
                    frame[y_min_l:y_max_l, x_min_l:x_max_l],
                    np.expand_dims(hand_det_results_l[0]["keypoints"], 0),
                    pose_kpt_color=[(0, 0, 255)]*21,
                    radius=4,
                    skeleton=skeleton,
                    pose_link_color=[(0, 0, 0)]*20
                )
                frame[y_min:y_max, x_min:x_max] = rendered
                frame[y_min_l:y_max_l, x_min_l:x_max_l] = rendered_l
                frame = cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 3)
                frame = cv2.rectangle(frame, (int(x_min_l), int(y_min_l)), (int(x_max_l), int(y_max_l)), (255, 0, 0), 3)
                
                if (writer is None):
                    writer = create_writer(frame, args, video.fps)
                writer.write(frame)
                
            hand_det_results_r[0]["keypoints"][:, 0] += x_min
            hand_det_results_r[0]["keypoints"][:, 1] += y_min
            hand_det_results_l[0]["keypoints"][:, 0] += x_min_l
            hand_det_results_l[0]["keypoints"][:, 1] += y_min_l

            
            pose_det_results[0]["keypoints"][112:112+21] = hand_det_results_r[0]["keypoints"]
            pose_det_results[0]["keypoints"][91:91+21] = hand_det_results_l[0]["keypoints"]
            
            hand_det_results_list.append(pose_det_results)
            if (frame_idx % 100 == 0): print(f"Idx: {frame_idx}, len: {len(video)}")
            frame_idx += 1
        with open(f'work_dirs/tumeke_testing/pickle_files/{args.video_name}_hands.p', 'wb') as outfile:
            pickle.dump(hand_det_results_list, outfile)
    else:
        with open(args.detections_hand_2d, 'rb') as f:
            hand_det_results_list = pickle.load(f)
                
    if (should_render_2d): writer.release()
    
    hand_lift_model = init_pose_model(
        args.hand_lifter_config,
        args.hand_lifter_checkpoint,
        device=args.device.lower())
    
    hand_lift_dataset = hand_lift_model.cfg.data['test']['type']
#    writer.release()
    # load temporal padding config from model.data_cfg
#     if hasattr(hand_lift_model.cfg, 'test_data_cfg'):
#         data_cfg = hand_lift_model.cfg.test_data_cfg
#     else:
#         data_cfg = hand_lift_model.cfg.data_cfg
    
#     if (isinstance(data_cfg, list)):
#         data_cfg = data_cfg[0]

#     for i, hand_det_results in enumerate(
#             mmcv.track_iter_progress(hand_det_results_list)):
#         # extract and pad input pose2d sequence
#         hands_results_2d = extract_pose_sequence(
#             hand_det_results_list,
#             frame_idx=i,
#             causal=data_cfg.causal,
#             seq_len=data_cfg.seq_len,
#             step=data_cfg.seq_frame_interval)
#         print("Len", len(hands_results_2d))
#         # 2D-to-3D pose lifting
#         hand_lift_results = inference_pose_lifter_model(
#             hand_lift_model,
#             pose_results_2d=hands_results_2d,
#             dataset=hand_lift_dataset,
#             with_track_id=False,
#             image_size=video.resolution,
#             norm_pose_2d=args.norm_pose_2d,
#             output_num=i,
#             dataset_info=hand_lift_model.cfg.dataset_info
#         )
        
#         # Pose processing
#         hand_lift_results_vis = []
#         for idx, res in enumerate(hand_lift_results):
#             keypoints_3d = res['keypoints_3d']
#             # exchange y,z-axis, and then reverse the direction of x,z-axis
#             keypoints_3d = keypoints_3d[..., [0, 2, 1]]
#             keypoints_3d[..., 0] = -keypoints_3d[..., 0]
#             keypoints_3d[..., 2] = -keypoints_3d[..., 2]
#             # rebase height (z-axis)
#             if args.rebase_keypoint_height:
#                 keypoints_3d[..., 2] -= np.min(
#                     keypoints_3d[..., 2], axis=-1, keepdims=True)
#             res['keypoints_3d'] = keypoints_3d
#             # add title
#             det_res = hand_det_results[idx]
#             res['title'] = f'Prediction ({idx})'
#             # only visualize the target frame
#             res['keypoints'] = det_res['keypoints']
#             res['bbox'] = det_res['bbox']
#             hand_lift_results_vis.append(res)

#         # Visualization
#         if num_instances < 0:
#             num_instances = len(hand_lift_results_vis)
#         img_vis = vis_3d_pose_result(
#             hand_lift_model,
#             result=hand_lift_results_vis,
#             img=video[i],
#             out_file=None,
#             radius=args.radius,
#             thickness=args.thickness,
#             num_instances=num_instances,
#         dataset=hand_lift_dataset)
        
#         if save_out_video:
#             if writer is None:
#                 writer = cv2.VideoWriter(
#                     osp.join(args.out_video_root,
#                              f'tumeke_testing/vis_hands_3d_{osp.basename(args.file_path)}'), fourcc,
#                     fps, (img_vis.shape[1], img_vis.shape[0]))
#             writer.write(img_vis)
    
    if save_out_video:
        writer.release()

In [4]:
args.video_name = 'wrist_rom'
args.file_path = 'demo/resources/test_vids/wrist_rom.mp4'
args.detections_2d = ''
args.detections_hand_2d = ''
process_video_hands(args)

Stage #1: 2D pose detection.
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_256x192-643e18cb_20200922.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/hand/hrnetv2/hrnetv2_w18_coco_wholebody_hand_256x256-1c028db7_20210908.pth
Initialized Model
Idx: 0, len: 836
Idx: 100, len: 836
Idx: 200, len: 836
Idx: 300, len: 836
Idx: 400, len: 836
Idx: 500, len: 836
Idx: 600, len: 836
Idx: 700, len: 836
Idx: 800, len: 836
load checkpoint from local path: /home/ubuntu/PoseEstimation/mmpose/tools/work_dirs/handpose3d_dex_27frames_fullconv_supervised/epoch_80.pth
